In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [6]:
i = -0.4
j = -0.1
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'correlation': 0,
            'subgroup_col_name': 'SEX',
            'marginal_prior': 1,
            'corr_btn_sens_and_output_per_subgroup': (i, j),
            # 'fixed_corr_in_test_data': True
    }, shortname = f"Corr_btn_sens_and_output_for_male_({i})_for_female_({j})"
)

[0, 1]
{0: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}, 1: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}}


100%|██████████| 2/2 [00:01<00:00,  1.64it/s]


{0: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}, 1: {(0, 1): 7500, (0, 0): 5000, (1, 1): 5000, (1, 0): 7500}}


100%|██████████| 2/2 [00:01<00:00,  1.67it/s]


In [4]:
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'subgroup_col_name': 'ST',
    }, shortname = f"Corr_btn_sens_and_output_for_ST_ranging_from_0_to_-0.5"
)

  2%|▏         | 1/51 [00:01<00:58,  1.17s/it]

before scaling: 505 392 1.288265306122449
after scaling: 392 392 1.0
before scaling: 496 125 3.968
after scaling: 125 125 1.0


  6%|▌         | 3/51 [00:03<00:52,  1.09s/it]

before scaling: 486 253 1.9209486166007905
after scaling: 253 253 1.0


 16%|█▌        | 8/51 [00:08<00:49,  1.16s/it]

...

before scaling: 265 171 1.5497076023391814
after scaling: 171 171 1.0


 98%|█████████▊| 50/51 [00:52<00:01,  1.03s/it]

before scaling: 746 326 2.2883435582822087
after scaling: 326 326 1.0


100%|██████████| 51/51 [00:53<00:00,  1.06s/it]

before scaling: 255 87 2.9310344827586206
after scaling: 87 87 1.0
[1000, 1000, 776, 252, 1000, 1000, 1000, 520, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 469, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 751, 1000, 371, 1000, 1000, 1000, 1000, 1000, 735, 1000, 710, 1000, 1000, 1000, 1000, 952, 400, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 715, 1000, 1000, 746, 337, 1000, 508, 1000, 1000, 1000, 821, 1000, 1000, 1000, 905, 1000, 1000, 1000, 1000, 738, 359, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 841, 1000, 1000, 509, 253, 1000, 1000, 1000, 1000, 1000, 1000, 450, 360, 1000, 1000, 1000, 1000, 947, 645, 1000, 1000, 436, 341]


In [46]:
tmp = pd.read_csv('~/Downloads/tabular/<PATH_TO_DATASET>/census19.csv')

In [47]:
tmp[['MAR']].value_counts()

MAR
0      915732
4      518315
2      179530
1       35485
3       26951
Name: count, dtype: int64

In [5]:
save_model=True
print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf_only_on_test = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    base_model = model_utils.get_model(max_iter=500)
    experiment.clf_only_on_test = copy.deepcopy(base_model)
    experiment.clf_only_on_test.fit(experiment.X_test, experiment.y_te_onehot)

    if save_model:
        model_utils.save_model(experiment.clf_only_on_test, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')

Training classifier for experiment: Census19_subgroup_col_name_ST


In [6]:
save_model = True

print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    experiment.clf = model_utils.get_model(max_iter=500)
    experiment.clf.fit(experiment.X_train, experiment.y_tr_onehot)

    if save_model:
        model_utils.save_model(experiment.clf, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model.pkl')

Training classifier for experiment: Census19_subgroup_col_name_ST
Loaded classifier for experiment from file: Census19_subgroup_col_name_ST


In [15]:

experiment.clf = model_utils.get_model(max_iter=500)
experiment.clf.fit(experiment.X_train, experiment.y_tr_onehot)

MLPClassifier(hidden_layer_sizes=(32, 16, 8), max_iter=500, random_state=42)

In [25]:
original_test_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==1].copy().reset_index(drop=True).drop(['is_train'], axis=1)

p = [0] * 51
n = [original_test_df[original_test_df['ST']==i][['MAR', 'PINCP']].value_counts().to_numpy().min() * 4 for i in range(51)]

temp_indices = experiment.ds.ds.sample_data_matching_correlation(original_test_df, p=p, n=n, subgroup_col_name='ST', transformed_already=True, return_indices_only=True)

experiment.X_train_balanced_corr = experiment.X_train.loc[temp_indices].reset_index(drop=True)
experiment.y_tr_balanced_corr = experiment.y_tr[temp_indices]
experiment.y_tr_onehot_balanced_corr = experiment.y_tr_onehot[temp_indices]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
[2000, 500, 1960, 1012, 1920, 1900, 1880, 876, 1840, 1820, 1800, 1344, 656, 1744, 1724, 1256, 1200, 1664, 1644, 652, 1604, 1588, 1568, 1548, 1096, 1508, 504, 748, 1448, 1176, 1408, 1264, 1372, 1352, 480, 1312, 1292, 1272, 1252, 1040, 1212, 304, 1176, 1156, 1136, 404, 1096, 1076, 684, 1036, 348]
{0: {(0, 1): 500, (0, 0): 500, (1, 1): 500, (1, 0): 500}, 1: {(0, 1): 125, (0, 0): 125, (1, 1): 125, (1, 0): 125}, 2: {(0, 1): 490, (0, 0): 490, (1, 1): 490, (1, 0): 490}, 3: {(0, 1): 253, (0, 0): 253, (1, 1): 253, (1, 0): 253}, 4: {(0, 1): 480, (0, 0): 480, (1, 1): 480, (1, 0): 480}, 5: {(0, 1): 475, (0, 0): 475, (1, 1): 475, (1, 0): 475}, 6: {(0, 1): 470, (0, 0): 470, (1, 1): 470, (1, 0): 470}, 7: {(0, 1): 219, (0, 0): 219, (1, 1): 219, (1, 0): 219}, 8: {(0, 1): 460, (0, 0): 460, (1, 1): 460, (1, 0): 4

100%|██████████| 51/51 [00:01<00:00, 41.17it/s]


In [35]:
base_model = model_utils.get_model(max_iter=500)
experiment.clf_balanced_corr = copy.deepcopy(base_model)
experiment.clf_balanced_corr.fit(experiment.X_train_balanced_corr, experiment.y_tr_onehot_balanced_corr)

MLPClassifier(hidden_layer_sizes=(32, 16, 8), max_iter=500, random_state=42)

In [11]:
subgroup_oh_cols = [f'ST_{i}' for i in range(51)]
subgroup_vals_tr = experiment.X_test[subgroup_oh_cols].to_numpy().argmax(axis=1)

In [12]:
subgroup_vals_tr

array([ 5, 21, 33, ..., 47,  9, 18])

In [39]:
def get_perf(experiment, clf, X_test, y_te, X_train, y_tr,  
    subgroup_col_name = 'ST', indices=None):
    y_tr_pred = np.argmax(clf.predict_proba(X_train), axis=1)

    subgroup_oh_cols = [f'ST_{i}' for i in range(51)]
    subgroup_vals_tr = X_train[subgroup_oh_cols].to_numpy().argmax(axis=1)
    subgroup_vals_te = X_test[subgroup_oh_cols].to_numpy().argmax(axis=1)
    # subgroup_vals_tr = X_train[f'{subgroup_col_name}_1'].to_numpy().ravel().astype(int)
    # subgroup_vals_te = X_test[f'{subgroup_col_name}_1'].to_numpy().ravel()

    sens_pred, case_indices = CSMIA_attack(clf, X_test, y_te, experiment.ds.ds.meta)
    sens_pred_LOMIA = LOMIA_attack(experiment, clf, X_test, y_te, experiment.ds.ds.meta, indices=indices)
    correct_indices = (sens_pred == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())
    correct_indices_LOMIA = (sens_pred_LOMIA == X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())

    # male_indices = np.where(subgroup_vals_tr==0)[0]
    # female_indices = np.where(subgroup_vals_tr==1)[0]
    
    return [correct_indices[subgroup_vals_te==i].mean() for i in range(51)]
    perf_dict = {
        'ASRD_CSMIA': round(100 * np.ptp([correct_indices[subgroup_vals_te==i].mean() for i in range(51)]), 2),
        'ASRD_LOMIA': round(100 * np.ptp([correct_indices_LOMIA[subgroup_vals_te==i].mean() for i in range(51)]), 2),
        'EOD': round(equalized_odds_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'DPD': round(demographic_parity_difference(y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr), 4),
        'MA': 100 * accuracy_score(y_tr.ravel()[:], y_tr_pred[:])
        # 'MA_Male': 100 * accuracy_score(y_tr.ravel()[male_indices], y_tr_pred[male_indices]),
        # 'MA_Female': 100 * accuracy_score(y_tr.ravel()[female_indices], y_tr_pred[female_indices])
    }

    return perf_dict


In [40]:
asrs = get_perf(experiment, experiment.clf, experiment.X_train, experiment.y_tr, experiment.X_test, experiment.y_te)

In [42]:
np.argmax(asrs)

49

In [48]:
asrs_balanced = get_perf(experiment, experiment.clf_balanced_corr, experiment.X_train_balanced_corr, experiment.y_tr_balanced_corr, experiment.X_test, experiment.y_te, indices=temp_indices)

In [51]:
experiment.X_train_balanced_corr.shape[0]/experiment.X_train.shape[0]

0.702852047948749

In [49]:
100 * np.max(asrs)

73.8

In [50]:
100 * np.max(asrs_balanced)

62.916666666666664

In [18]:
np.ptp(asrs)

0.22838256746900076

In [88]:
model_arch = hidden_layer_size_tuples[1]
res_dict = {
    'w/o BCorr': get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr),
    'w Bcorr': get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr, indices=temp_indices)
}
res_dict_df = pd.DataFrame.from_dict(res_dict, orient='index')

In [89]:
res_dict_df

,ASRD_CSMIA,ASRD_LOMIA,EOD,DPD,MA_Male,MA_Female
w/o BCorr,12.52,7.48,0.0674,0.0178,73.208,78.16
w Bcorr,2.06,2.28,0.0416,0.0015,73.888,77.90


In [90]:
print(res_dict_df.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
 & ASRD_CSMIA & ASRD_LOMIA & EOD & DPD & MA_Male & MA_Female \\
\midrule
w/o BCorr & 12.520000 & 7.480000 & 0.067400 & 0.017800 & 73.208000 & 78.160000 \\
w Bcorr & 2.060000 & 2.280000 & 0.041600 & 0.001500 & 73.888000 & 77.900000 \\
\bottomrule
\end{tabular}



In [64]:
res_dict

{'w/o BCorr': {'MA_Female': 78.16,
  'MA_Male': 73.208,
  'EOD': 0.06735999999999998,
  'DPD': 0.017839999999999967,
  'ASRD': 0.12524000000000002},
 'w Bcorr': {'MA_Female': 77.9,
  'MA_Male': 73.888,
  'EOD': 0.04160000000000008,
  'DPD': 0.0014800000000000368,
  'ASRD': 0.020580823232929313}}

In [61]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs[model_arch], experiment.X_test, experiment.y_te, experiment.X_train, experiment.y_tr))

{'MA_Female': 77.164, 'MA_Male': 72.712, 'EOD': 0.057999999999999996, 'DPD': 0.013480000000000047, 'ASRD': 0.08372000000000002}
{'MA_Female': 78.16, 'MA_Male': 73.208, 'EOD': 0.06735999999999998, 'DPD': 0.017839999999999967, 'ASRD': 0.12524000000000002}
{'MA_Female': 78.99199999999999, 'MA_Male': 73.628, 'EOD': 0.06672, 'DPD': 0.013080000000000092, 'ASRD': 0.14991999999999994}
{'MA_Female': 75.932, 'MA_Male': 72.348, 'EOD': 0.03735999999999995, 'DPD': 0.0015199999999999658, 'ASRD': 0.05384}


In [62]:
for model_arch in experiment.clfs:
    print(get_perf(experiment, experiment.clfs_balanced_corr[model_arch], experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, experiment.X_train, experiment.y_tr))

{'MA_Female': 76.728, 'MA_Male': 72.272, 'EOD': 0.04927999999999999, 'DPD': 0.0047200000000000575, 'ASRD': 0.01446057842313686}
{'MA_Female': 77.9, 'MA_Male': 73.888, 'EOD': 0.04160000000000008, 'DPD': 0.0014800000000000368, 'ASRD': 0.020580823232929313}
{'MA_Female': 78.532, 'MA_Male': 75.312, 'EOD': 0.049360000000000015, 'DPD': 0.017159999999999953, 'ASRD': 0.004560182407296276}
{'MA_Female': 75.62, 'MA_Male': 72.38799999999999, 'EOD': 0.03527999999999998, 'DPD': 0.0029599999999999627, 'ASRD': 0.0006000240009600422}


In [12]:
get_corr_btn_sens_and_out_per_subgroup(experiment, experiment.X_test_balanced_corr, experiment.y_te_balanced_corr, {'SEX': 0})

0.10008400408079707